In [1]:
using FunctionalCollections
using Iterators
using Pipe
using Compat

macro printval(ee)
    ee_expr = @sprintf "%s" string(ee)
    esc(:(println($ee_expr," = ", $ee)))
end

macro pz(ee)
    ee_expr = @sprintf "%s" string(ee)
    esc(:(println($ee_expr,"\t\t",typeof($ee), "\t", size($ee))))
end

push!(LOAD_PATH, ".")
push!(LOAD_PATH, "../util/")
using DataStructures
using DataStructuresExtended
using JLD
using DataFrames

In [2]:
function groupby_value(fn::Function, xs)
    key_type = typeof(fn(first(xs)))
    group_type = Vector{typeof(first(xs))}
    groups = Dict{key_type, group_type}()
    
    for x in xs
        key = fn(x)
        group = get!(()->group_type(), groups, key)
        push!(group, x)
    end
    groups
end

groupby_value (generic function with 1 method)

In [3]:
OUTPUT_DIR = "../../documents_prepared/Journal\ Papers/SumResynthesis/data/"

"../../documents_prepared/Journal Papers/SumResynthesis/data/"

In [4]:
function jaccard_index(aa::DataStructures.Accumulator,bb::DataStructures.Accumulator)
    sum(intersect(aa,bb))/sum(union(aa,bb))
end

function jaccard_index(aa,bb)
    length(intersect(aa,bb))/length(union(aa,bb))
end

jaccard_index (generic function with 2 methods)

In [5]:
function precision(actual::DataStructures.Accumulator,reference::DataStructures.Accumulator)
    sum(intersect(actual,reference))/sum(actual)
end
function recall(actual::DataStructures.Accumulator,reference::DataStructures.Accumulator)
    sum(intersect(actual,reference))/sum(reference)
end

recall (generic function with 1 method)

In [10]:

function flatten_blocked_records{T<:Number, V}(records::Dict{T,V})  
    rec_types = @pipe records |> values |> map(typeof,_) |> union
    rec_type = length(rec_types)==1 ? rec_types[1] : V
    ret = rec_type() #Construct it, it is some form of vector, or we will error soon
    sizehint!(ret) = sum([length(r) for r in values(records)])
    
    sorted_keys = @pipe records |> keys |> collect |> sort
    for key in sorted_keys
        push!(ret, records[key]...)
    end
    ret
end

function flatten_raw_blocked_records{T,V}(records::Dict{T,V})  
    key_types = @pipe records |> keys|> map(parse,_) |> map(typeof,_) |> union
    @assert length(key_types)==1
    key_type =  key_types[1]
    
    rec_types = @pipe records |> values |> map(typeof,_) |> union
    rec_type = length(rec_types)==1 ? rec_types[1] : V
    flatten_blocked_records(Dict(Pair{key_type, rec_type}[parse(k)=>v for (k,v) in records]))
end

flatten_raw_blocked_records (generic function with 1 method)

In [11]:

#books_glove300cityblock_raw = load("results/bags/books_corpus_0.001_of_test_glove300_cityblock.jld","4")
books_glove300eculidean_raw = flatten_raw_blocked_records(load("results/bags/books_corpus_0.001_of_test_glove300.jld"))
#brown_glove300_raw = load("results/bags/brown_glove300_res.jld", "res")
#brown_glove200_raw = load("results/bags/brown_glove200_res.jld", "res")
#brown_glove100_raw = load("results/bags/brown_glove100_res.jld", "res")
#brown_glove50_raw = load("results/bags/brown_glove50_res.jld", "res")

6677-element Array{Tuple{Array{ASCIIString,1},Array{ASCIIString,1},Float32},1}:
 (ASCIIString["``","you","would","destroy","a","book","that","even","the","quran","calls","holy","?","''"],ASCIIString["``","you","holy","would","book","that","a","quran","destroy","calls","the","even","?","''"],-2.5348536f-6)                                                                                                                                                             
 (ASCIIString["im","glad","that","i","left","henderson","."],ASCIIString["i","left","im","that","henderson","glad","."],-1.4708529f-6)                                                                                                                                                                                                                                                         
 (ASCIIString["he","ran","around","the","student","apartments","yelling","fire",",","fire","!"],ASCIIString["fire","fire","he","!","around","ran","apart

In [ ]:
@assert all([rset[1] for rset in brown_glove50_raw] .== [rset[1] for rset in brown_glove300_raw])
@assert all([rset[1] for rset in brown_glove50_raw] .== [rset[1] for rset in brown_glove200_raw])
@assert all([rset[1] for rset in brown_glove50_raw] .== [rset[1] for rset in brown_glove100_raw])

In [14]:
bag_res = DataFrame()
bag_res[:ground] = [counter(rset[1]) for rset in books_glove300eculidean_raw]
bag_res[:ground_len] = Int[sum(ss) for ss in bag_res[:ground]]
exp_names = ASCIIString[]
for raw_name in ["books_glove300eculidean_raw"]#"brown_glove50_raw","brown_glove100_raw", "brown_glove200_raw", "brown_glove300_raw"]
    colname = split(raw_name,"_")[2] 
    push!(exp_names, colname)
    raw = eval(raw_name|>Symbol)
    bag_res[colname*"_actual"|> Symbol] = [counter(rset[2]) for rset in raw]
    #bag_res[colname*"_distance"|> Symbol] = Float64[-1*rset[3] for rset in raw]
end

#bag_res

In [21]:
for exp_name in exp_names
    actual = bag_res[exp_name*"_actual" |> symbol]
    bag_res[exp_name*"_perfect" |> symbol] = actual.==bag_res[:ground]
    bag_res[exp_name*"_jaccard" |> symbol] = map(jaccard_index, actual, bag_res[:ground])
    bag_res[exp_name*"_precision" |> symbol] = map(precision, actual, bag_res[:ground])
    bag_res[exp_name*"_recall" |> symbol] = map(recall, actual, bag_res[:ground])
end

In [22]:
bag_res.colindex |> keys

7-element Array{Symbol,1}:
 :ground                     
 :ground_len                 
 :glove300eculidean_actual   
 :glove300eculidean_perfect  
 :glove300eculidean_jaccard  
 :glove300eculidean_precision
 :glove300eculidean_recall   

In [23]:
keep_cols = @pipe (bag_res.colindex |> keys |> map(string,_)
                    |>filter(col -> contains(col,"jaccard") ||  contains(col,"perfect") ||  contains(col,"recall") ||  contains(col,"precision"),_)
                    |> map(Symbol,_))
len_scores = aggregate(bag_res[[:ground_len, keep_cols...]], :ground_len, mean)
sort!(len_scores)
#writetable(OUTPUT_DIR*"selection_len_scores.csv", len_scores)
len_scores

,ground_len,glove300eculidean_perfect_mean,glove300eculidean_jaccard_mean,glove300eculidean_precision_mean,glove300eculidean_recall_mean
1,1,1.0,1.0,1.0,1.0
2,2,1.0,1.0,1.0,1.0
3,3,1.0,1.0,1.0,1.0
4,4,1.0,1.0,1.0,1.0
5,5,0.9871134020618557,0.9929123711340206,0.9946244477172312,0.9953608247422681
6,6,0.9932279909706546,0.9977426636568849,0.9981188863807374,0.999247554552295
7,7,0.9734299516908212,0.9874731615673644,0.9912870945479642,0.9917184265010351
8,8,0.9558139534883721,0.9831289640591966,0.9882364341085271,0.9906976744186047
9,9,0.9644549763033176,0.9881008407074758,0.9919359471492174,0.993417588204318
10,10,0.905511811023622,0.9716519957835381,0.9790628881180062,0.9868766404199475


In [ ]:
[:ground_len, keep_cols...]

In [ ]:
cum_len_scores = DataFrame()
cum_len_scores[:max_ground_len] = bag_res[:ground_len] |> unique |> sort!
for jaccard_col in jaccard_cols
    cum_len_scores[jaccard_col] = Float64[ begin
            shortish_sents = bag_res[:ground_len] .<= max_len;
            bag_res[shortish_sents, jaccard_col] |> mean
         end for max_len in cum_len_scores[:max_ground_len]]
end
for perfect_col in perfect_cols
    cum_len_scores[perfect_col] = Float64[ begin
            shortish_sents = bag_res[:ground_len] .<= max_len;
            bag_res[shortish_sents, perfect_col] |> mean
         end for max_len in cum_len_scores[:max_ground_len]]
end
#writetable(OUTPUT_DIR*"selection_cum_len_scores.csv", cum_len_scores)
cum_len_scores 


In [ ]:
cum_len_scores[57,:]

In [ ]:
function longform(df::DataFrame, index::Symbol, cols::Vector{Symbol}, new_name::Symbol)
    function subdf(col::Symbol)
        df_sub = df[[index, col]]
        rename!(df_sub, col, new_name)
        df_sub[:label] = col
        df_sub
    end
    vcat([subdf(col) for col in cols]...)
end




In [ ]:
using Gadfly

In [ ]:
p=Gadfly.plot(longform(cum_len_scores, :max_ground_len, jaccard_cols, :mean_jaccard_index),
    x="max_ground_len", y="mean_jaccard_index",
    color="label",
    Coord.Cartesian(ymin=0),
    Geom.point
)
#draw(PNG("brown_cumulative_length_score.png", 24cm, 6cm), p)

In [ ]:

p=Gadfly.plot(longform(len_scores, :ground_len, [string(col)*"_mean"|> Symbol for col in jaccard_cols], :mean_jaccard_index),
    x="ground_len", y="mean_jaccard_index",
    color="label",
    Geom.point
)
#draw(PNG("brown_length_score.png", 24cm, 6cm), p)

In [ ]:
bag_res[:ground_len] |> describe 
p=Gadfly.plot(bag_res, x="ground_len",  Geom.histogram)
#draw(PNG("brown_length_dist.png", 24cm, 6cm), p)

In [ ]:
`git add ../../documents_prepared/Journal\ Papers/SumResynthesis/data/ordering_scores.csv` |> run
`git commit -m="new results` |> run
`git push` |> run

In [ ]:
##############################################
# ORDERED
########################################

In [ ]:
using PyCall
@pyimport nltk
@pyimport nltk.align.bleu_score as nltk_bleu

function bleu_score(candidate, reference)
    reference = reference |> collect
    candidate = candidate |> collect
    
    if reference==candidate #Perfect Match
        1.0
    else
        weights = [1,1,1,1]/4
        nltk_bleu.bleu(candidate,Any[reference], weights)
    end
end

In [ ]:
bleu_score(["the","comedy","of","manners","of","this","first","a","basis","is","performed","in","1892","."],
           ["this","is","the" ,"basis","of","a" ,"comedy" ,"of","manners","first","performed","in","1892", "."]
)

In [ ]:
immutable OrderRecord{T}
    reference::Vector{T}
    actual::Vector{T}
    prob::Float64
    time::Float64
    status::Symbol
end

In [ ]:
books_0_001 = flatten_blocked_records(Dict{Int,Any}([key=>load("results/ordered/books_corpus_0.001_of_test_glove300.jld",string(key)) for key in 1:100]))
books_0_001 = map(r->OrderRecord(r...), books_0_001)

In [ ]:
#folds50 = load("results/ordered/brown_glove50_ordered_vA1000.jld")
#folds100 = load("results/ordered/brown_glove100_ordered_vA1000.jld")
#folds200 = load("results/ordered/brown_glove200_ordered_vA1000.jld")
folds300_loglikelyhood_A5000_2 = load("results/ordered/brown_glove300_ordered_vA5000_loglikelyhood_2.jld")
folds300_loglikelyhood_A5000 = load("results/ordered/brown_glove300_ordered_vA5000_loglikelyhood.jld")
#folds300_loglikelyhood_A1000 = load("results/ordered/brown_glove300_ordered_vA1000_loglikelyhood.jld")
#folds300 = load("results/ordered/brown_glove300_ordered_vA1000.jld")
#folds_oracle = load("results/ordered/brown_oracle_ordered_vA1000.jld")\

#folds_oracle_loglikelyhood_A5000 = load("results/ordered/brown_oracle_ordered_vA5000_loglikelyhood.jld")
#folds_oracle_loglikelyhood_A5000_2 = load("results/ordered/brown_oracle_ordered_vA5000_loglikelyhood_2.jld")
#folds_oracle_loglikelyhood_A10000_2 = load("results/ordered/brown_oracle_ordered_vA10000_loglikelyhood_2.jld")
#folds_oracle_loglikelyhood_A1000 = load("results/ordered/brown_oracle_ordered_vA1000_Loglikelyhood.jld")
#all_ordered_exp_folds = Any[folds_oracle, folds_oracle_loglikelyhood,folds300,folds300_loglikelyhood]#[folds50,folds100,folds200,folds300,folds_oracle]
all_ordered_exp_folds=Any[folds300_loglikelyhood_A5000,folds300_loglikelyhood_A5000_2]


In [ ]:
attempted_only{T<:OrderRecord}(recs::Vector{T}) = filter(r->r.status!=:NotAttempted, recs)
feasible_only{T<:OrderRecord}(recs::Vector{T}) = filter(r->r.status!=:Infeasible, recs)


In [ ]:
function exact_match(rec::OrderRecord)
    rec.reference == rec.actual
end
function bleu_score(rec::OrderRecord)
    bleu_score( rec.actual, rec.reference)
end

In [ ]:
funct

In [ ]:
@pipe books_0_001 |> attempted_only |>  map(exact_match,_) |> mean

In [ ]:
@pipe books_0_001 |> attempted_only |> feasible_only |> map(exact_match,_) |> mean

In [ ]:
@pipe books_0_001 |> attempted_only |> map(bleu_score,_) |> mean

In [ ]:
@pipe books_0_001 |> attempted_only |> feasible_only |> map(bleu_score,_) |> mean

In [ ]:
function mean_by_length{T<:OrderRecord}(rater::Function, data::Vector{T})
    groups = groupby_value(r->length(r.reference), data) 
    Dict([len=>mean(map(rater,group)) for (len, group) in groups])
end

In [ ]:
@pipe mean_by_length(r->r.status==:Optimal, books_0_001|> attempted_only) |> collect|> sort! 

In [ ]:
@pipe mean_by_length(exact_match, books_0_001|> attempted_only |>feasible_only) |> collect|> sort! 


In [ ]:
@pipe mean_by_length(bleu_score, books_0_001|> attempted_only |>feasible_only) |> collect|> sort! 


In [ ]:
@pipe mean_by_length(r->r.time , books_0_001|> attempted_only) |> collect|> sort! 

In [ ]:
attempted_only{T}(fold::Vector{T}) = filter(eap->!isnan(eap[3]), fold)
nongiveup_only{T}(fold::Vector{T}) = filter(eap->0<eap[3]<1, fold)
partial_only{T}(fold::Vector{T}) = filter(eap->eap[3]>1, fold)

attempted_only{T}(folds::Vector{Vector{T}}) = map(attempted_only, folds)
nongiveup_only{T}(folds::Vector{Vector{T}}) = map(nongiveup_only, folds)
partial_only{T}(folds::Vector{Vector{T}}) = map(partial_only, folds)

function get_bleu_score(folds)
    bleu_scores = map(keys(folds), values(folds)) do fold_name,res
        fold_bleu_scores = Float64[]
        for (reference, candidate, prob) in res
            push!(fold_bleu_scores, bleu_score(candidate, reference))
        end
        fold_bleu_scores
    end
end

In [ ]:
get_bleu_score(books_0_001)

In [ ]:
bleu_scores = map(all_ordered_exp_folds) do folds
    bleu_scores = get_bleu_score(folds);
    map(mean, bleu_scores) |> mean
end

In [ ]:
bleu_scores_no_giveup = map(all_ordered_exp_folds) do folds
    mean([mean([bleu_score(eap[1], eap[2]) for eap in nongiveup_only(attempted_only(fold))]) for fold in values(folds)])
end

In [ ]:
giveups = map(all_ordered_exp_folds) do folds
    mean([mean(Bool[!(0<eap[3]<1) for eap in attempted_only(fold)]) for fold in values(folds)])
end

In [ ]:
exact_matchs = map(all_ordered_exp_folds) do folds
    mean([mean(Bool[eap[1]==eap[2] for eap in attempted_only(fold)]) for fold in values(folds)])
end

In [ ]:
partials = map(all_ordered_exp_folds) do folds
    mean([mean(Bool[eap[3]>1 for eap in attempted_only(fold)]) for fold in values(folds)])
end

In [ ]:
exact_matchs_no_giveup = map(all_ordered_exp_folds) do folds
    mean([mean(Bool[eap[1]==eap[2] for eap in nongiveup_only(attempted_only(fold))]) for fold in values(folds)])
end

In [ ]:
ordering_res = DataFrame(Model=["Oracle", "Oracle_LL", "300d", 300d_LL])#["50d","100d","200d","300d", "Oracle"])
ordering_res[symbol("BLEU Score")]=bleu_scores
ordering_res[symbol("Exact Ordered Matches")]=exact_matchs
ordering_res[symbol("Giveups")]=giveups
ordering_res[symbol("BLEU Score Excluding Giveups")]=bleu_scores_no_giveup
ordering_res[symbol("Exact Ordered Matches Excluding Giveups")]=exact_matchs_no_giveup
#writetable(OUTPUT_DIR*"ordering_scores.csv", ordering_res)
ordering_res

In [ ]:
########

In [ ]:
fold_indexes=load("brown_glove_folds.jld","fold_indexes");

In [ ]:
function fold(data, fold_indexes)
    ret = Dict()
    for (fold_number,fold_iis) in enumerate(fold_indexes)
        ret["fold_$(fold_number)"] = data[fold_iis]
    end
    ret
end

In [ ]:
jaccard_folds_300 = fold(bag_res[:glove300_jaccard], fold_indexes)

In [ ]:
[jaccard_folds_300["fold_6"] bleu_scores300[1]]

In [ ]:
get_bleu_score(folds300,  count_give_ups=true, count_nonattepts=true);